# Gibt es „Angstgegner“-Nationen, gegen die die deutschen Spieler besonders schlecht abschneiden? Lässt sich das einfach durch die generelle Spielstärke der Spieler dieser Nationen erklären, oder gibt es Nationen, gegen die gerade deutsche Spieler schlecht abschneiden? Gibt es analog „Lieblingsgegner“-Nationen?

In [1]:
import json
import pandas as pd
import plotly.express as px

In [2]:
matches = pd.read_csv('../data/raw/atp_matches_till_2022.csv')
players = pd.read_csv('../data/raw/atp_players_till_2022.csv')

In [3]:
with open('../data/processed/länder.json', 'r') as f:
    data = json.load(f)

codes = list(data.keys())
names = list(data.values())

länder = pd.DataFrame({'Code': codes, 'Name': names})

In [35]:
german_ioc_code = "GER"
german_winner_matches = matches[matches["winner_ioc"] == german_ioc_code]
german_loser_matches = matches[matches["loser_ioc"] == german_ioc_code]

siege = german_winner_matches["loser_ioc"].value_counts()
niederlagen = german_loser_matches["winner_ioc"].value_counts()

performance = pd.DataFrame({"Siege": siege, "Niederlagen": niederlagen}).fillna(0)

performance["Total"] = performance.sum(axis=1)
performance["Win_Rate"] = performance["Siege"] / performance["Total"]
performance["Loss_Rate"] = performance["Niederlagen"] / performance["Total"]

Grenze = 50

performance = performance[performance["Total"] >= Grenze].sort_values(by="Win_Rate", ascending=False)
performance = pd.concat([performance, länder.set_index('Code')], axis=1, join='inner')

top = 10
top_favoriten = performance.head(top)
top_rivalen = performance.tail(top)

In [36]:
performance.sort_values(by="Win_Rate", ascending=True).head(10)

,Siege,Niederlagen,Total,Win_Rate,Loss_Rate,Name
MAR,25.0,43.0,68.0,0.367647,0.632353,Marokko
SUI,169.0,245.0,414.0,0.408213,0.591787,Schweiz
CHI,87.0,122.0,209.0,0.416268,0.583732,Chile
SRB,95.0,133.0,228.0,0.416667,0.583333,Serbien
PER,21.0,29.0,50.0,0.420000,0.580000,Peru
ESP,691.0,945.0,1636.0,0.422372,0.577628,Spanien
ARG,358.0,478.0,836.0,0.428230,0.571770,Argentinien
POL,40.0,53.0,93.0,0.430108,0.569892,Polen
CRO,189.0,248.0,437.0,0.432494,0.567506,Kroatien
RUS,307.0,392.0,699.0,0.439199,0.560801,Russland


In [37]:
print(top_favoriten)
print()
print(top_rivalen.sort_values(by="Loss_Rate",ascending=False))

     Siege  Niederlagen  Total  Win_Rate  Loss_Rate          Name
DEN   55.0         33.0   88.0  0.625000   0.375000      Dänemark
MEX   41.0         25.0   66.0  0.621212   0.378788        Mexiko
POR   31.0         19.0   50.0  0.620000   0.380000      Portugal
BLR   32.0         21.0   53.0  0.603774   0.396226  Weißrussland
KAZ   35.0         25.0   60.0  0.583333   0.416667    Kasachstan
IND   60.0         43.0  103.0  0.582524   0.417476        Indien
COL   44.0         32.0   76.0  0.578947   0.421053     Kolumbien
RSA  215.0        163.0  378.0  0.568783   0.431217     Südafrika
JPN   79.0         60.0  139.0  0.568345   0.431655         Japan
ITA  343.0        292.0  635.0  0.540157   0.459843       Italien

     Siege  Niederlagen   Total  Win_Rate  Loss_Rate         Name
MAR   25.0         43.0    68.0  0.367647   0.632353      Marokko
SUI  169.0        245.0   414.0  0.408213   0.591787      Schweiz
CHI   87.0        122.0   209.0  0.416268   0.583732        Chile
SRB   95.

In [28]:
fig = px.bar(performance.sort_values('Win_Rate', ascending=False), 
            y=['Win_Rate', 'Loss_Rate'], 
            title='Gewinn- und Verlustrate der deutschen Tennisspieler gegen andere Nationen',
            hover_data=['Siege', 'Niederlagen'],
            color_discrete_sequence=["#b2e061", "#fd7f6f"],
            opacity=0.9)

fig.add_hline(y=0.5, opacity=0.4, line_dash='dot', line_color='black')

fig.add_annotation(text="50% Gewinnrate",
                xref='paper',
                y=performance['Win_Rate'].median(),
                arrowhead=1, 
                showarrow=True, 
                arrowcolor='black',        
                bgcolor='white',
                font=dict(size=10, color='grey'))


fig.update_yaxes(tickformat=".2%")
fig.update_yaxes(title='Gewinnrate')
fig.update_xaxes(title='Gegner Nationen')
fig.show()


In [38]:
matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188156,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,4,209362,NaN,NaN,...,68.0,42.0,12.0,10.0,11.0,13.0,1103.0,9.0,1130.0,8.0
188157,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,5,202065,NaN,NaN,...,56.0,40.0,20.0,15.0,4.0,8.0,808.0,23.0,1390.0,4.0
188158,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,1,138846,NaN,NaN,...,54.0,29.0,8.0,11.0,6.0,10.0,1059.0,10.0,1881.0,1.0
188159,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,2,209409,NaN,NaN,...,39.0,24.0,7.0,10.0,5.0,9.0,NaN,NaN,NaN,NaN
